[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/RF.ipynb)

# RandomForest(회귀 모델)

In [ ]:
# Colab용 notebook입니다. 이 notebook 한 장에서 여러 데이터의 다운로드부터, 추천까지 완결하도록 되어 있습니다(예측 평가는 미포함)
# MovieLens 데이터를 아직 다운로드 하지 않았다면, 이 셀을 실행해서 다운로드합니다.
# MovieLens 데이터 분석은 data_download.ipynb를 참조합니다.

# 데이터 다운로드와 압축 풀기
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

--2022-12-27 05:42:50--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘../data/ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  64.4MB/s    in 1.0s    

2022-12-27 05:42:51 (64.4 MB/s) - ‘../data/ml-10m.zip’ saved [65566137/65566137]

Archive:  ../data/ml-10m.zip
   creating: ../data/ml-10M100K/
  inflating: ../data/ml-10M100K/allbut.pl  
  inflating: ../data/ml-10M100K/movies.dat  
  inflating: ../data/ml-10M100K/ratings.dat  
  inflating: ../data/ml-10M100K/README.html  
  inflating: ../data/ml-10M100K/split_ratings.sh  
  inflating: ../data/ml-10M100K/tags.dat  


In [ ]:
# Movielens 데이터 로딩(데이터량이 많으므로, 로딩에 시간이 걸릴 수 있습니다)
import pandas as pd

# movieID와 제목만 사용
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genre를 list 형식으로 저장한다
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# 사용자가 부여한 영화의 태그 정보를 로딩한다
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tag를 소문자로 바꾼다
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tag를 영화별로 list 형식으로 저장한다
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# 태그 정보를 결합한다
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 평갓값 데이터만 로딩한다
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# 데이터량이 많으므로 사용자수를 1000으로 줄여서 시험해본다
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 영화 데이터와 평가 데이터를 결합한다
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 학습용과 데이터용으로 데이터를 나눈다
# 각 사용자의 최근 5건의 영화를 평가용으로 사용하고, 나머지는 학습용으로 사용한다
# 우선, 각 사용자가 평가한 영화의 순서를 계산한다
# 최근 부여한 영화부터 순서를 부여한다(1에서 시작)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [ ]:
# 평갓값을 사용자 x 영화 행렬로 변환한다. 결손값은 평균값 또는 0으로 채운다
user_movie_matrix = movielens_train.pivot(index="user_id", columns="movie_id", values="rating")
user_id2index = dict(zip(user_movie_matrix.index, range(len(user_movie_matrix.index))))
movie_id2index = dict(zip(user_movie_matrix.columns, range(len(user_movie_matrix.columns))))
user_movie_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,62000,62113,62293,62344,62394,62801,62803,63113,63992,64716
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,5.0,NaN,3.0,NaN,3.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 학습에 사용하는 학습용 데이터 안의 사용자와 영화의 조합을 얻는다
train_keys = movielens_train[["user_id", "movie_id"]]
# 학습용 데이터 안의 평갓값을 학습의 정답 데이터로 얻는다
train_y = movielens_train.rating.values

# 평갓값을 예측할 테스트용 데이터 안의 사용자와 영화의 조합을 얻는다
test_keys = movielens_test[["user_id", "movie_id"]]
# 순위 형식의 추천 리스트 작성을 위해 학습용 데이터에 존재하는 모든 사용자와 모든 영화의 조합을 얻는다
train_all_keys = user_movie_matrix.stack(dropna=False).reset_index()[["user_id", "movie_id"]]


In [ ]:
# 특징량을 작성한다
train_x = train_keys.copy()
test_x = test_keys.copy()
train_all_x = train_all_keys.copy()

In [ ]:
# 학습용 데이터에 존재하는 사용자별 평갓값의 최솟값, 최댓값, 평균값
# 및, 영화별 평갓값의 최솟값, 최댓값, 평균값을 특징량으로 추가한다
aggregators = ["min", "max", "mean"]
user_features = movielens_train.groupby("user_id").rating.agg(aggregators).to_dict()
movie_features = movielens_train.groupby("movie_id").rating.agg(aggregators).to_dict()
for agg in aggregators:
    train_x[f"u_{agg}"] = train_x["user_id"].map(user_features[agg])
    test_x[f"u_{agg}"] = test_x["user_id"].map(user_features[agg])
    train_all_x[f"u_{agg}"] = train_all_x["user_id"].map(user_features[agg])
    train_x[f"m_{agg}"] = train_x["movie_id"].map(movie_features[agg])
    test_x[f"m_{agg}"] = test_x["movie_id"].map(movie_features[agg])
    train_all_x[f"m_{agg}"] = train_all_x["movie_id"].map(movie_features[agg])
# 테스트용 데이터에만 존재하는 사용자나 영화의 특징량을, 학습용 데이터 전체의 평균 평갓값으로 채운다
average_rating = train_y.mean()
test_x.fillna(average_rating, inplace=True)

In [ ]:
import itertools

# 영화가 특정한 genre인지 나타내는 특징량을 추가한다
movie_genres = movies[["movie_id", "genre"]]
genres = set(list(itertools.chain(*movie_genres.genre)))
for genre in genres:
    movie_genres[f"is_{genre}"] = movie_genres.genre.apply(lambda x: genre in x)
movie_genres.drop("genre", axis=1, inplace=True)
train_x = train_x.merge(movie_genres, on="movie_id")
test_x = test_x.merge(movie_genres, on="movie_id")
train_all_x = train_all_x.merge(movie_genres, on="movie_id")

<ipython-input-7-eae16a73cde7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_genres[f"is_{genre}"] = movie_genres.genre.apply(lambda x: genre in x)


In [ ]:
# 특징량으로 사용하지 않는 정보는 삭제한다
train_x = train_x.drop(columns=["user_id", "movie_id"])
test_x = test_x.drop(columns=["user_id", "movie_id"])
train_all_x = train_all_x.drop(columns=["user_id", "movie_id"])

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR

# Random Forest를 사용한 학습
reg = RFR(n_jobs=-1, random_state=0)
reg.fit(train_x.values, train_y)


RandomForestRegressor(n_jobs=-1, random_state=0)

In [ ]:
# 테스트용 데이터 안의 사용와 영화의 조합에 대해 평갓값을 예측한다
test_pred = reg.predict(test_x.values)

movie_rating_predict = test_keys.copy()
movie_rating_predict["rating_pred"] = test_pred

In [ ]:
from collections import defaultdict
import numpy as np

# 학습용 데이터에 존재하는 모든 사용자와 모든 영화의 조합에 대해 평갓값을 예측한다
train_all_pred = reg.predict(train_all_x.values)

pred_train_all = train_all_keys.copy()
pred_train_all["rating_pred"] = train_all_pred
pred_matrix = pred_train_all.pivot(index="user_id", columns="movie_id", values="rating_pred")

# 사용자가 학습용 데이터 안에서 평가하지 않은 영화 중에서
# 예측 평갓값이 높은 순으로 10편의 영화를 순위 형식으로 추천 리스트로 만든다
pred_user2items = defaultdict(list)
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()
for user_id in movielens_train.user_id.unique():
    movie_indexes = np.argsort(-pred_matrix.loc[user_id, :]).values
    for movie_index in movie_indexes:
        movie_id = user_movie_matrix.columns[movie_index]
        if movie_id not in (user_evaluated_movies[user_id]):
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break
pred_user2items

defaultdict(list,
            {139: [7986, 6356, 6565, 4270, 6538, 5817, 4828, 3588, 631, 26974],
             149: [5914, 1672, 1570, 555, 5928, 616, 640, 5996, 1326, 1671],
             182: [4956,
              5971,
              6729,
              8379,
              6713,
              5662,
              30894,
              6890,
              7647,
              5090],
             215: [5598, 4626, 4679, 6286, 4458, 4802, 4887, 848, 3274, 5572],
             281: [8482, 1497, 715, 880, 1483, 4531, 6583, 5472, 6722, 1992],
             326: [2269, 2211, 2393, 1084, 1643, 1711, 1336, 1186, 1156, 523],
             351: [6312, 6058, 7458, 7117, 7523, 7340, 6008, 8485, 7836, 8369],
             357: [56949,
              56587,
              59037,
              13,
              2569,
              2243,
              2993,
              2013,
              56801,
              126],
             426: [3718, 3035, 3620, 7817, 3892, 3387, 3496, 3223, 3546, 5723],
             45

In [ ]:
# user_id=2인 사용자가 학습 데이터에 평가를 부여한 영화 목록
movielens_train[movielens_train.user_id==2]

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
4732,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0
5246,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0
5798,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0
6150,2,648,2.0,868244699,Mission: Impossible (1996),"[Action, Adventure, Mystery, Thriller]","[confusing, confusing plot, memorable sequence...",12.0
6531,2,733,3.0,868244562,"Rock, The (1996)","[Action, Adventure, Thriller]","[gfei own it, alcatraz, nicolas cage, sean con...",18.0
6813,2,736,3.0,868244698,Twister (1996),"[Action, Adventure, Romance, Thriller]","[disaster, disaster, storm, bill paxton, helen...",13.0
7113,2,780,3.0,868244698,Independence Day (a.k.a. ID4) (1996),"[Action, Adventure, Sci-Fi, War]","[action, alien invasion, aliens, will smith, a...",14.0
7506,2,786,3.0,868244562,Eraser (1996),"[Action, Drama, Thriller]","[arnold schwarzenegger, action, arnold, arnold...",19.0
7661,2,802,2.0,868244603,Phenomenon (1996),"[Drama, Romance]","[interesting concept, own, john travolta, john...",15.0
7779,2,858,2.0,868245645,"Godfather, The (1972)","[Crime, Drama]","[oscar (best picture), marlon brando, classic,...",9.0


In [ ]:
pred_user2items[2]

[4210, 4961, 4105, 2351, 3573, 4032, 1586, 5159, 4840, 2804]

In [ ]:
# user_id=2에 대한 추천(4210, 4961, 4105)
movies[movies.movie_id.isin([4210, 4961, 4105])]

,movie_id,title,genre,tag
4013,4105,"Evil Dead, The (1981)","[Fantasy, Horror]","[directorial debut, bruce campbell, cult class..."
4118,4210,Manhunter (1986),"[Action, Crime, Drama, Horror, Thriller]","[hannibal lecter, serial killer, ei muista, er..."
4867,4961,Pornstar: The Legend of Ron Jeremy (2001),[Documentary],[pornography]
